# HW5 codes
members:<br>
1. 劉恩兆 B14081027
2. 宋穎恩 B14081302

注意事項：為留下每次上傳的模型參數設定，會出現較多相似的cell

# loading packages & setting

In [ ]:
# !pip install spacy
# !python -m spacy download en_core_web_sm
# !pip install torch
# !pip install transformers

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
# import wordcloud
# from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
nltk.download('stopwords')
import spacy
nlp = spacy.load('en_core_web_sm')
from tqdm import tqdm
stop_words = stopwords.words('english')
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\danny\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danny\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# loading data

In [15]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.shape

(14869, 2)

In [4]:
test.shape

(9914, 2)

In [50]:
train

,class,tweet
0,1,"[9-1-13] 2:50 pm ""son of a bitch ate my mac n ..."
1,1,RT @BryceSerna: Don't be a pussy grab the boot...
2,2,RT @ClicquotSuave: bunch of rappers boutta flo...
3,2,@michigannews13 wow. Thats great language comi...
4,1,"and this is why I'm single, I don't fuck with ..."
...,...,...
14864,1,RT @Estellleeee: Lol bitches act hard when the...
14865,1,RT @yes_paul: I bitch about unnecessary things...
14866,1,"RT @Im_Amy_Bitches: I'm not always a bitch, so..."
14867,1,RT @dkiswinning: GamePlan: 1. Eat the pussy 2....


In [8]:
test

,id,tweet
0,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
2,2,"!!!!!!""@__BrighterDays: I can not just sit up ..."
3,3,!!!!&#8220;@selfiequeenbri: cause I'm tired of...
4,4,""" @rhythmixx_ :hobbies include: fighting Maria..."
...,...,...
9909,9909,you're all niggers
9910,9910,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
9911,9911,young buck wanna eat!!.. dat nigguh like I ain...
9912,9912,youu got wild bitches tellin you lies


#### 將資料分出 input & label

In [18]:
# input為['tweet']
train_inputs = train['tweet']
test_inputs = test['tweet']
# label為train資料的['class']
train_labels = train['class']

# TF-IDF

##### 清除無用無字以及停用詞

In [16]:
sw = stopwords.words('english')
lemmatizer = WordNetLemmatizer() 

def clean_text(text):
    
    text = text.lower()
    
    text = re.sub(r"^(rt)", "", text) #Removing RT in the beginning 
    
    text = re.sub(r"(@[a-zA-Z0-9_]*[:;, ])", "", text) #Removing @user 
    
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text) # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")

    text = re.sub(r"http\S+", "",text) #Removing URLs 
    #text = re.sub(r"http", "",text)
    
    html=re.compile(r'<.*?>') 
    
    text = html.sub(r'',text) #Removing html tags
    
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations
        
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    
    text = [lemmatizer.lemmatize(word) for word in text]
    
    text = " ".join(text) #removing stopwords
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) #Removing emojis
    
    text = re.sub(r"^(am)", "",text)
    text = re.sub(r"^(pm)", "",text)
    
    return text

In [17]:
train['tweet'] = train['tweet'].apply(lambda x: clean_text(x))
test['tweet'] = test['tweet'].apply(lambda x: clean_text(x))
train.head(30)

,class,tweet
0,1,son bitch ate mac n cheese http tco ojyz w
1,1,pussy grab booty love booty appreciate booty
2,2,bunch rapper boutta flood internet w trash rem...
3,2,"wow thats great language coming h coach, sure ..."
4,1,"single, fuck bitch attitude foh"
5,1,yea hoe rocking friday last night
6,1,free da real niccas n give snitch da time
7,0,ur fag
8,1,judge appointed obama doesnt see agenda fuckin...
9,2,wonder nsa spook following


##### 將文字資料轉為TF-IDF矩陣

In [19]:
tfidf_vectorizer = TfidfVectorizer() 

tfidf_train_vectors = tfidf_vectorizer.fit_transform(train_inputs)

tfidf_test_vectors = tfidf_vectorizer.transform(test_inputs)

##### Random Forest

In [11]:
classifier = RandomForestClassifier()

classifier.fit(tfidf_train_vectors, train_labels)

RandomForestClassifier()

In [12]:
y_pred = classifier.predict(tfidf_test_vectors)

In [13]:
prediction_df = pd.DataFrame(y_pred)
id = pd.DataFrame([*range(0,9914)])
upload = pd.concat([id, prediction_df], axis=1)
upload.columns=['id','class']
upload.to_csv('upload_12.csv')
upload

,id,class
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
9909,9909,1
9910,9910,1
9911,9911,1
9912,9912,1


##### LightGBM

In [41]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
classifier = lgb.LGBMClassifier(n_estimators=200,max_depth=5)
classifier.fit(tfidf_train_vectors, train_labels)
y_pred = classifier.predict(tfidf_test_vectors)

In [44]:
prediction_df = pd.DataFrame(y_pred)
id = pd.DataFrame([*range(0,9914)])
upload = pd.concat([id, prediction_df], axis=1)
upload.columns=['id','class']
upload.to_csv('upload_30.csv')
upload

,id,class
0,0,1
1,1,2
2,2,1
3,3,1
4,4,1
...,...,...
9909,9909,0
9910,9910,2
9911,9911,1
9912,9912,1


##### MLP

In [18]:
from sklearn.neural_network import MLPClassifier
classifier = MLPClassifier(solver='sgd', activation='relu',alpha=1e-4,hidden_layer_sizes=(50,50), random_state=1,max_iter=20,verbose=10,learning_rate_init=.1)
classifier.fit(tfidf_train_vectors, train_labels)
y_pred = classifier.predict(tfidf_test_vectors)

Iteration 1, loss = 0.68004319
Iteration 2, loss = 0.58427757
Iteration 3, loss = 0.40228923
Iteration 4, loss = 0.33145592
Iteration 5, loss = 0.28448658
Iteration 6, loss = 0.25869075
Iteration 7, loss = 0.27971799
Iteration 8, loss = 0.31371892
Iteration 9, loss = 0.35723937
Iteration 10, loss = 0.28098913
Iteration 11, loss = 0.17647206
Iteration 12, loss = 0.12655612
Iteration 13, loss = 0.09796744
Iteration 14, loss = 0.63822520
Iteration 15, loss = 0.29286453
Iteration 16, loss = 0.23689069
Iteration 17, loss = 0.40922554
Iteration 18, loss = 0.18151037
Iteration 19, loss = 0.26444635
Iteration 20, loss = 0.20654021


C:\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


In [19]:
prediction_df = pd.DataFrame(y_pred)
id = pd.DataFrame([*range(0,9914)])
upload = pd.concat([id, prediction_df], axis=1)
upload.columns=['id','class']
upload.to_csv('upload_14.csv')
upload

,id,class
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
9909,9909,1
9910,9910,2
9911,9911,1
9912,9912,1


# Bert Model

### check GPU

In [2]:
import torch
import torch.nn as nn
# If there's a GPU available
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If there's no GPU available
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
    
# 使用GPU來訓練

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3070 Laptop GPU


### Dataset settings

In [21]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {0:0,
          1:1,
          2:2 }
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = [labels[label] for label in df['class']]
        self.texts = [tokenizer(text, 
                                padding='max_length', 
                                max_length = 512, 
                                truncation=True,
                                return_tensors="pt") 
                      for text in df['tweet']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

### construct a model

##### upload 1-5 setting

In [24]:
#upload 1-5 setting
from transformers import BertModel

class BertClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 3)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

##### upload 7 setting

In [23]:
#upload 7 setting
from transformers import BertModel

class BertClassifier(nn.Module):
    def __init__(self, dropout=0.3):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 3)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

### training the model

##### upload 1-5 setting

In [25]:
from torch.optim import Adam
from tqdm import tqdm

def trainmodel(model, train_data, learning_rate, epochs):
  # 通過Dataset類獲取訓練集
    train = Dataset(train_data)
    # DataLoader根據batch_size獲取數據，訓練時選擇打亂樣本
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
  # 判斷是否使用GPU
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    # 定義損失函數和優化器
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()
    # 開始進入訓練循環
    for epoch_num in range(epochs):
      # 定義兩個變量，用於存儲訓練集的準確率和損失
            total_acc_train = 0
            total_loss_train = 0
      # 進度條函數tqdm
            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
        # 通過模型得到輸出
                output = model(input_id, mask)
                # 計算損失
                #
                train_label = train_label.type(torch.LongTensor)
                train_label = train_label.to(device)
                #
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                # 計算精度
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc
        # 模型更新
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            print(
                f'''Epochs: {epoch_num + 1} 
              | Train Loss: {total_loss_train / len(train_data): .3f} 
              | Train Accuracy: {total_acc_train / len(train_data): .3f}''')

In [61]:
EPOCHS = 10
model = BertClassifier()
LR = 1e-6
trainmodel(model, train, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████| 7435/7435 [12:14<00:00, 10.12it/s]


Epochs: 1 
              | Train Loss:  0.223 
              | Train Accuracy:  0.844


100%|██████████████████████████████████████████████████████████████████████████████| 7435/7435 [12:13<00:00, 10.13it/s]


Epochs: 2 
              | Train Loss:  0.139 
              | Train Accuracy:  0.913


100%|██████████████████████████████████████████████████████████████████████████████| 7435/7435 [12:09<00:00, 10.20it/s]


...

Epochs: 8 
              | Train Loss:  0.017 
              | Train Accuracy:  0.991


100%|██████████████████████████████████████████████████████████████████████████████| 7435/7435 [12:16<00:00, 10.09it/s]


Epochs: 9 
              | Train Loss:  0.011 
              | Train Accuracy:  0.994


100%|██████████████████████████████████████████████████████████████████████████████| 7435/7435 [12:19<00:00, 10.05it/s]

Epochs: 10 
              | Train Loss:  0.007 
              | Train Accuracy:  0.996


In [26]:
#20
EPOCHS = 12
model = BertClassifier()
LR = 1e-6
trainmodel(model, train, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████| 7435/7435 [12:09<00:00, 10.19it/s]


Epochs: 1 
              | Train Loss:  0.228 
              | Train Accuracy:  0.835


100%|██████████████████████████████████████████████████████████████████████████████| 7435/7435 [12:04<00:00, 10.26it/s]


Epochs: 2 
              | Train Loss:  0.138 
              | Train Accuracy:  0.911


100%|██████████████████████████████████████████████████████████████████████████████| 7435/7435 [12:01<00:00, 10.31it/s]


...

Epochs: 10 
              | Train Loss:  0.009 
              | Train Accuracy:  0.994


100%|██████████████████████████████████████████████████████████████████████████████| 7435/7435 [12:01<00:00, 10.31it/s]


Epochs: 11 
              | Train Loss:  0.008 
              | Train Accuracy:  0.994


100%|██████████████████████████████████████████████████████████████████████████████| 7435/7435 [12:01<00:00, 10.31it/s]

Epochs: 12 
              | Train Loss:  0.007 
              | Train Accuracy:  0.995


##### upload 7 setting

In [16]:
from torch.optim import Adam
from tqdm import tqdm

def trainmodel(model, train_data, learning_rate, epochs):
  # 通過Dataset類獲取訓練集
    train = Dataset(train_data)
    # DataLoader根據batch_size獲取數據，訓練時選擇打亂樣本
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=6, shuffle=True)
  # 判斷是否使用GPU
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    # 定義損失函數和優化器
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()
    # 開始進入訓練循環
    for epoch_num in range(epochs):
      # 定義兩個變量，用於存儲訓練集的準確率和損失
            total_acc_train = 0
            total_loss_train = 0
      # 進度條函數tqdm
            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
        # 通過模型得到輸出
                output = model(input_id, mask)
                # 計算損失
                #
                train_label = train_label.type(torch.LongTensor)
                train_label = train_label.to(device)
                #
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                # 計算精度
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc
        # 模型更新
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            print(
                f'''Epochs: {epoch_num + 1} 
              | Train Loss: {total_loss_train / len(train_data): .3f} 
              | Train Accuracy: {total_acc_train / len(train_data): .3f}''')

In [8]:
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
trainmodel(model, train, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████| 3718/3718 [09:46<00:00,  6.34it/s]


Epochs: 1 
              | Train Loss:  0.118 
              | Train Accuracy:  0.836


100%|██████████████████████████████████████████████████████████████████████████████| 3718/3718 [09:43<00:00,  6.37it/s]


Epochs: 2 
              | Train Loss:  0.072 
              | Train Accuracy:  0.910


100%|██████████████████████████████████████████████████████████████████████████████| 3718/3718 [09:41<00:00,  6.39it/s]


Epochs: 3 
              | Train Loss:  0.059 
              | Train Accuracy:  0.921


100%|██████████████████████████████████████████████████████████████████████████████| 3718/3718 [09:41<00:00,  6.39it/s]


Epochs: 4 
              | Train Loss:  0.048 
              | Train Accuracy:  0.933


100%|██████████████████████████████████████████████████████████████████████████████| 3718/3718 [09:41<00:00,  6.40it/s]

Epochs: 5 
              | Train Loss:  0.038 
              | Train Accuracy:  0.943


##### upload 8 setting

In [13]:
EPOCHS = 2
model = BertClassifier()
LR = 1e-6
trainmodel(model, train, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████| 3718/3718 [09:44<00:00,  6.36it/s]


Epochs: 1 
              | Train Loss:  0.111 
              | Train Accuracy:  0.857


100%|██████████████████████████████████████████████████████████████████████████████| 3718/3718 [09:49<00:00,  6.30it/s]

Epochs: 2 
              | Train Loss:  0.072 
              | Train Accuracy:  0.911


##### upload 9 setting

In [8]:
EPOCHS = 12
model = BertClassifier()
LR = 1e-6
trainmodel(model, train, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████| 2479/2479 [09:08<00:00,  4.52it/s]


Epochs: 1 
              | Train Loss:  0.081 
              | Train Accuracy:  0.832


100%|██████████████████████████████████████████████████████████████████████████████| 2479/2479 [09:01<00:00,  4.58it/s]


Epochs: 2 
              | Train Loss:  0.047 
              | Train Accuracy:  0.912


100%|██████████████████████████████████████████████████████████████████████████████| 2479/2479 [09:08<00:00,  4.52it/s]


...

Epochs: 10 
              | Train Loss:  0.005 
              | Train Accuracy:  0.994


100%|██████████████████████████████████████████████████████████████████████████████| 2479/2479 [09:04<00:00,  4.56it/s]


Epochs: 11 
              | Train Loss:  0.004 
              | Train Accuracy:  0.995


100%|██████████████████████████████████████████████████████████████████████████████| 2479/2479 [09:03<00:00,  4.56it/s]

Epochs: 12 
              | Train Loss:  0.003 
              | Train Accuracy:  0.995


In [17]:
EPOCHS = 10
model = BertClassifier()
LR = 0.0001
trainmodel(model, train, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████| 2479/2479 [09:06<00:00,  4.53it/s]


Epochs: 1 
              | Train Loss:  0.115 
              | Train Accuracy:  0.771


100%|██████████████████████████████████████████████████████████████████████████████| 2479/2479 [09:05<00:00,  4.55it/s]


Epochs: 2 
              | Train Loss:  0.114 
              | Train Accuracy:  0.773


100%|██████████████████████████████████████████████████████████████████████████████| 2479/2479 [09:01<00:00,  4.58it/s]


...

Epochs: 8 
              | Train Loss:  0.111 
              | Train Accuracy:  0.773


100%|██████████████████████████████████████████████████████████████████████████████| 2479/2479 [09:03<00:00,  4.56it/s]


Epochs: 9 
              | Train Loss:  0.111 
              | Train Accuracy:  0.773


100%|██████████████████████████████████████████████████████████████████████████████| 2479/2479 [09:01<00:00,  4.58it/s]

Epochs: 10 
              | Train Loss:  0.112 
              | Train Accuracy:  0.773


### model predicting

In [27]:
class_names = [0,1,2]
prediction_list = []
tweet = test['tweet'].tolist()
# tweet

In [28]:
for i in tweet:
    encoded_review = tokenizer.encode_plus(
        i,
        truncation=True,
        max_length=512,
        add_special_tokens=True,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    _, prediction = torch.max(output, dim=1)
    prediction_list.append(class_names[prediction])

C:\Anaconda\lib\site-packages\transformers\tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [29]:
prediction_df = pd.DataFrame(prediction_list)
id = pd.DataFrame([*range(0,9914)])
upload = pd.concat([id, prediction_df], axis=1)
upload.columns=['id','class']
upload.to_csv('upload_20.csv')
upload

,id,class
0,0,1
1,1,0
2,2,1
3,3,1
4,4,1
...,...,...
9909,9909,0
9910,9910,2
9911,9911,1
9912,9912,1


## No punctuation version

In [29]:
train_no_punc = train.copy()
test_no_punc = test.copy()
train_no_punc['tweet'] = train_no_punc['tweet'].str.replace(r'[^\w\s]+', '')
test_no_punc['tweet'] = test_no_punc['tweet'].str.replace(r'[^\w\s]+', '')
train_no_punc
# input為['tweet']
train_inputs = train['tweet']
test_inputs = test['tweet']
# label為train資料的['class']
train_labels = train['class']

C:\Users\danny\AppData\Local\Temp\ipykernel_23900\2102231581.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train_no_punc['tweet'] = train_no_punc['tweet'].str.replace(r'[^\w\s]+', '')
C:\Users\danny\AppData\Local\Temp\ipykernel_23900\2102231581.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  test_no_punc['tweet'] = test_no_punc['tweet'].str.replace(r'[^\w\s]+', '')


### Dataset settings

In [30]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {0:0,
          1:1,
          2:2 }
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = [labels[label] for label in df['class']]
        self.texts = [tokenizer(text, 
                                padding='max_length', 
                                max_length = 512, 
                                truncation=True,
                                return_tensors="pt") 
                      for text in df['tweet']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

### construct a model

In [31]:
#11
from transformers import BertModel

class BertClassifier(nn.Module):
    def __init__(self, dropout=0.3):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 3)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

### training the model

##### upload 11 setting

In [34]:
from torch.optim import Adam
from tqdm import tqdm

def trainmodel(model, train_data, learning_rate, epochs):
  # 通過Dataset類獲取訓練集
    train = Dataset(train_data)
    # DataLoader根據batch_size獲取數據，訓練時選擇打亂樣本
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=7, shuffle=True)
  # 判斷是否使用GPU
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    # 定義損失函數和優化器
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()
    # 開始進入訓練循環
    for epoch_num in range(epochs):
      # 定義兩個變量，用於存儲訓練集的準確率和損失
            total_acc_train = 0
            total_loss_train = 0
      # 進度條函數tqdm
            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
        # 通過模型得到輸出
                output = model(input_id, mask)
                # 計算損失
                #
                train_label = train_label.type(torch.LongTensor)
                train_label = train_label.to(device)
                #
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                # 計算精度
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc
        # 模型更新
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            print(
                f'''Epochs: {epoch_num + 1} 
              | Train Loss: {total_loss_train / len(train_data): .3f} 
              | Train Accuracy: {total_acc_train / len(train_data): .3f}''')

In [35]:
EPOCHS = 10
model = BertClassifier()
LR = 1e-6
trainmodel(model, train, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████| 2125/2125 [09:22<00:00,  3.78it/s]


Epochs: 1 
              | Train Loss:  0.128 
              | Train Accuracy:  0.320


100%|██████████████████████████████████████████████████████████████████████████████| 2125/2125 [09:25<00:00,  3.76it/s]


Epochs: 2 
              | Train Loss:  0.059 
              | Train Accuracy:  0.767


100%|██████████████████████████████████████████████████████████████████████████████| 2125/2125 [09:21<00:00,  3.79it/s]


...

Epochs: 8 
              | Train Loss:  0.015 
              | Train Accuracy:  0.970


100%|██████████████████████████████████████████████████████████████████████████████| 2125/2125 [09:23<00:00,  3.77it/s]


Epochs: 9 
              | Train Loss:  0.011 
              | Train Accuracy:  0.978


100%|██████████████████████████████████████████████████████████████████████████████| 2125/2125 [09:20<00:00,  3.79it/s]

Epochs: 10 
              | Train Loss:  0.008 
              | Train Accuracy:  0.986


### model predicting

In [36]:
class_names = [0,1,2]
prediction_list = []
tweet = test['tweet'].tolist()
# tweet

In [37]:
for i in tweet:
    encoded_review = tokenizer.encode_plus(
        i,
        truncation=True,
        max_length=512,
        add_special_tokens=True,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    _, prediction = torch.max(output, dim=1)
    prediction_list.append(class_names[prediction])

C:\Anaconda\lib\site-packages\transformers\tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [38]:
prediction_df = pd.DataFrame(prediction_list)
id = pd.DataFrame([*range(0,9914)])
upload = pd.concat([id, prediction_df], axis=1)
upload.columns=['id','class']
upload.to_csv('upload_11.csv')
upload

,id,class
0,0,1
1,1,0
2,2,1
3,3,1
4,4,1
...,...,...
9909,9909,0
9910,9910,2
9911,9911,1
9912,9912,1


## 清理乾淨版本

In [24]:
def clean_text(text):
    
    text = text.lower()
    
    text = re.sub(r"^(rt)", "", text) #Removing RT in the beginning 
    
    text = re.sub(r"(@[a-zA-Z0-9_]*[:;, ])", "", text) #Removing @user 
    
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text) # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")

    text = re.sub(r"http\S+", "",text) #Removing URLs 
    #text = re.sub(r"http", "",text)
    
    html=re.compile(r'<.*?>') 
    
    text = html.sub(r'',text) #Removing html tags
    
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations
    
    text = " ".join(text) #removing stopwords
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) #Removing emojis
    
    text = re.sub(r"^(am)", "",text)
    text = re.sub(r"^(pm)", "",text)
    
    return text

In [25]:
train_clean = train.copy()
test_clean = test.copy()
train_clean['tweet'] = train_clean['tweet'].apply(lambda x: clean_text(x))
test_clean['tweet'] = test_clean['tweet'].apply(lambda x: clean_text(x))
# input為['tweet']
train_inputs = train_clean['tweet']
test_inputs = test_clean['tweet']
# label為train資料的['class']
train_labels = train_clean['class']

### Dataset settings

In [26]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {0:0,
          1:1,
          2:2 }
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = [labels[label] for label in df['class']]
        self.texts = [tokenizer(text, 
                                padding='max_length', 
                                max_length = 512, 
                                truncation=True,
                                return_tensors="pt") 
                      for text in df['tweet']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

### construct a model

In [10]:
#17
from transformers import BertModel

class BertClassifier(nn.Module):
    def __init__(self, dropout=0.4):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 3)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

In [27]:
#19
from transformers import BertModel

class BertClassifier(nn.Module):
    def __init__(self, dropout=0.45):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 3)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

### training the model

##### upload 17 setting

In [31]:
from torch.optim import Adam
from tqdm import tqdm

def trainmodel(model, train_data, learning_rate, epochs):
  # 通過Dataset類獲取訓練集
    train = Dataset(train_data)
    # DataLoader根據batch_size獲取數據，訓練時選擇打亂樣本
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=6, shuffle=True)
  # 判斷是否使用GPU
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    # 定義損失函數和優化器
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()
    # 開始進入訓練循環
    for epoch_num in range(epochs):
      # 定義兩個變量，用於存儲訓練集的準確率和損失
            total_acc_train = 0
            total_loss_train = 0
      # 進度條函數tqdm
            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
        # 通過模型得到輸出
                output = model(input_id, mask)
                # 計算損失
                #
                train_label = train_label.type(torch.LongTensor)
                train_label = train_label.to(device)
                #
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                # 計算精度
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc
        # 模型更新
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            print(
                f'''Epochs: {epoch_num + 1} 
              | Train Loss: {total_loss_train / len(train_data): .3f} 
              | Train Accuracy: {total_acc_train / len(train_data): .3f}''')

In [12]:
EPOCHS = 10
model = BertClassifier()
LR = 1e-6
trainmodel(model, train, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████| 2125/2125 [09:31<00:00,  3.72it/s]


Epochs: 1 
              | Train Loss:  0.081 
              | Train Accuracy:  0.750


100%|██████████████████████████████████████████████████████████████████████████████| 2125/2125 [09:25<00:00,  3.76it/s]


Epochs: 2 
              | Train Loss:  0.047 
              | Train Accuracy:  0.900


100%|██████████████████████████████████████████████████████████████████████████████| 2125/2125 [09:24<00:00,  3.77it/s]


...

Epochs: 8 
              | Train Loss:  0.016 
              | Train Accuracy:  0.962


100%|██████████████████████████████████████████████████████████████████████████████| 2125/2125 [09:19<00:00,  3.80it/s]


Epochs: 9 
              | Train Loss:  0.012 
              | Train Accuracy:  0.971


100%|██████████████████████████████████████████████████████████████████████████████| 2125/2125 [09:19<00:00,  3.80it/s]

Epochs: 10 
              | Train Loss:  0.009 
              | Train Accuracy:  0.979


##### upload 19 setting

In [32]:
#19
EPOCHS = 10
model = BertClassifier()
LR = 1e-5
trainmodel(model, train, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████| 2479/2479 [09:00<00:00,  4.58it/s]


Epochs: 1 
              | Train Loss:  0.057 
              | Train Accuracy:  0.888


100%|██████████████████████████████████████████████████████████████████████████████| 2479/2479 [09:06<00:00,  4.54it/s]


Epochs: 2 
              | Train Loss:  0.041 
              | Train Accuracy:  0.916


100%|██████████████████████████████████████████████████████████████████████████████| 2479/2479 [09:08<00:00,  4.52it/s]


...

Epochs: 8 
              | Train Loss:  0.002 
              | Train Accuracy:  0.996


100%|██████████████████████████████████████████████████████████████████████████████| 2479/2479 [08:59<00:00,  4.59it/s]


Epochs: 9 
              | Train Loss:  0.002 
              | Train Accuracy:  0.996


100%|██████████████████████████████████████████████████████████████████████████████| 2479/2479 [08:58<00:00,  4.60it/s]

Epochs: 10 
              | Train Loss:  0.001 
              | Train Accuracy:  0.997


### model predicting

In [33]:
class_names = [0,1,2]
prediction_list = []
tweet = test['tweet'].tolist()
# tweet

In [34]:
for i in tweet:
    encoded_review = tokenizer.encode_plus(
        i,
        truncation=True,
        max_length=512,
        add_special_tokens=True,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    _, prediction = torch.max(output, dim=1)
    prediction_list.append(class_names[prediction])

C:\Anaconda\lib\site-packages\transformers\tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [35]:
prediction_df = pd.DataFrame(prediction_list)
id = pd.DataFrame([*range(0,9914)])
upload = pd.concat([id, prediction_df], axis=1)
upload.columns=['id','class']
upload.to_csv('upload_19.csv')
upload

,id,class
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
9909,9909,0
9910,9910,2
9911,9911,1
9912,9912,1


## padding=256測試

### Dataset settings

In [17]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {0:0,
          1:1,
          2:2 }
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = [labels[label] for label in df['class']]
        self.texts = [tokenizer(text, 
                                padding='max_length', 
                                max_length = 256, 
                                truncation=True,
                                return_tensors="pt") 
                      for text in df['tweet']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

### construct a model

##### upload 18 setting

In [18]:
from transformers import BertModel

class BertClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 3)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

### training the model

In [19]:
from torch.optim import Adam
from tqdm import tqdm

def trainmodel(model, train_data, learning_rate, epochs):
  # 通過Dataset類獲取訓練集
    train = Dataset(train_data)
    # DataLoader根據batch_size獲取數據，訓練時選擇打亂樣本
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=10, shuffle=True)
  # 判斷是否使用GPU
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    # 定義損失函數和優化器
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()
    # 開始進入訓練循環
    for epoch_num in range(epochs):
      # 定義兩個變量，用於存儲訓練集的準確率和損失
            total_acc_train = 0
            total_loss_train = 0
      # 進度條函數tqdm
            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
        # 通過模型得到輸出
                output = model(input_id, mask)
                # 計算損失
                #
                train_label = train_label.type(torch.LongTensor)
                train_label = train_label.to(device)
                #
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                # 計算精度
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc
        # 模型更新
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            print(
                f'''Epochs: {epoch_num + 1} 
              | Train Loss: {total_loss_train / len(train_data): .3f} 
              | Train Accuracy: {total_acc_train / len(train_data): .3f}''')

In [20]:
EPOCHS = 10
model = BertClassifier()
LR = 1e-6
trainmodel(model, train, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████| 1487/1487 [04:33<00:00,  5.44it/s]


Epochs: 1 
              | Train Loss:  0.060 
              | Train Accuracy:  0.763


100%|██████████████████████████████████████████████████████████████████████████████| 1487/1487 [04:40<00:00,  5.30it/s]


Epochs: 2 
              | Train Loss:  0.040 
              | Train Accuracy:  0.836


100%|██████████████████████████████████████████████████████████████████████████████| 1487/1487 [04:41<00:00,  5.29it/s]


...

Epochs: 8 
              | Train Loss:  0.015 
              | Train Accuracy:  0.937


100%|██████████████████████████████████████████████████████████████████████████████| 1487/1487 [04:24<00:00,  5.63it/s]


Epochs: 9 
              | Train Loss:  0.013 
              | Train Accuracy:  0.945


100%|██████████████████████████████████████████████████████████████████████████████| 1487/1487 [04:22<00:00,  5.66it/s]

Epochs: 10 
              | Train Loss:  0.011 
              | Train Accuracy:  0.953


### model predicting

In [21]:
class_names = [0,1,2]
prediction_list = []
tweet = test['tweet'].tolist()
# tweet

In [22]:
for i in tweet:
    encoded_review = tokenizer.encode_plus(
        i,
        truncation=True,
        max_length=256,
        add_special_tokens=True,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    _, prediction = torch.max(output, dim=1)
    prediction_list.append(class_names[prediction])

C:\Anaconda\lib\site-packages\transformers\tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [23]:
prediction_df = pd.DataFrame(prediction_list)
id = pd.DataFrame([*range(0,9914)])
upload = pd.concat([id, prediction_df], axis=1)
upload.columns=['id','class']
upload.to_csv('upload_18.csv')
upload

,id,class
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
9909,9909,1
9910,9910,2
9911,9911,1
9912,9912,1
